In [1]:
import geopandas as gpd
import requests
import json
from datetime import datetime
from sqlalchemy import create_engine
from tqdm import tqdm
tqdm.pandas()
import time
import flexpolyline as fp
from shapely import Polygon

D:\Temp\ipykernel_10008\1033793127.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [28]:
# читаем какой-то файл
df = gpd.read_file('C:/Users/apikuleva/Desktop/sokolniki_point1.shp')

In [29]:
df['geometry'].crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [21]:
df['geometry'] = df.geometry.to_crs(4326)

In [22]:
# функция для чтения того, что получили в запросе 
# Также можно прочитать файл из базы

# def select_pg(sql, geom):
#     return gpd.read_postgis(sql, engine, geom_col = geom, crs = 3857)

# engine = create_engine('postgresql+psycopg2://kb_geo:kYZQK90yvE8aNi54ELINc0yJ1gu6wo7h@\
# db03.cluster.strlk.ru/kb_graph')

# # считываем таблицу 
# df = select_pg(f'select * from baikal_vhods', 'geom')
# df['geom'] = df.geom.to_crs(4326)


KeyboardInterrupt: 

In [21]:
df

,Name,Shape_Leng,Shape_Area,geometry
0,Полигон 2,2660.728379,277264.323315,"MULTIPOLYGON (((83.78624 53.32741, 83.78591 53..."


In [6]:
df['geometry'].centroid

D:\Temp\ipykernel_10008\1722743555.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['geometry'].centroid


0    POINT (37.67247 55.80458)
dtype: geometry

In [30]:
apikey = 'A0ShEHBEXlqgtNWamgRa1upMZBD6Fdf6MZU9meDq1CU'

In [31]:
def get_iso(geom, distance, transport='pedestrian', mode='time', departureTime='any'):
    resp = requests.get("https://isoline.router.hereapi.com/v8/isolines?" +
         f"transportMode={transport}" +
         f"&origin={str(geom.y)},{str(geom.x)}" +
         ";radius=200" +  # для того, чтобы центроид снепался к нормальной дороге и не застревал в промке 
         f"&range[type]={mode}" + 
         f"&range[values]={distance}" +
         #f"&departureTime={departureTime}" + # с этой штукой надо что-то придумать, мб сделать здесь if типа если депарчер тайм не задан, то не включаем в запрос. Любой вариант депарчер тайма ведет к одному результату
         #f"&overrideFlowDuration={distance}" +                   
         f"&apikey={apikey}")
    resp_json = json.loads(resp.text)
    time.sleep(1)  # чтобы не забанили
    # return resp_json
    # return resp_json['isolines'][0]['polygons'][0]
    return resp_json['isolines'][0]['polygons'][0]['outer']

In [14]:
df

,osm_id,geometry,iso_15_pesh,iso_geom_15
0,1333996,"MULTIPOLYGON (((37.68802 55.81181, 37.68863 55...",BGw58tqDqun6nCrFgE1CiIAmgBzCiIrFsF1CgIAwV1CgIr...,"POLYGON ((37.65425 55.80226, 37.65431 55.80218..."


In [32]:
df['iso_15_pesh'] = df.geometry.centroid.progress_apply(lambda x: get_iso(x, 15*60, transport='pedestrian', departureTime='2023-04-10T18:00:00'))

D:\Temp\ipykernel_10008\3424886817.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['iso_15_pesh'] = df.geometry.centroid.progress_apply(lambda x: get_iso(x, 15*60, transport='pedestrian', departureTime='2023-04-10T18:00:00'))
  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:17<00:00,  1.79s/it]


In [34]:
# df['iso_15'] = df['geometry'].centroid.progress_apply(lambda x: get_iso(x, 15*60, departureTime='2022-11-10T18:00:00'))
# для пешеходных используем pedestrian
# df['iso_15_pesh'] = df.geometry.progress_apply(lambda x: get_iso(x, 20*60, transport='pedestrian', departureTime='2023-04-10T18:00:00'))
df['iso_geom_15'] = df['iso_15_pesh'].apply(lambda x: Polygon([(i[1], i[0]) for i in fp.decode(x)]))
df_15 = gpd.GeoDataFrame(df, geometry= 'iso_geom_15', crs="EPSG:4326")
df_15[['id','iso_geom_15']].to_file('df_151.geojson')

In [ ]:
df_15.explore()

In [17]:
df.explore()